In [ ]:
import os
import cv2
import time
import json
import pickle
import numpy as np
import face_recognition
import faiss

In [ ]:

import concurrent.futures

KNOWN_DIR = "known_faces/dataset_imgs"
ENCODING_FILE = "encodings_hog_faiss/encodings.pkl"
FAISS_INDEX_FILE = "encodings_hog_faiss/faiss.index"

MAX_WORKERS = 6
RESIZE_SCALE = 0.5
MATCH_THRESHOLD = 0.50


# ---------- Encode Single Image ----------
def encode_image(file):
    try:
        path = os.path.join(KNOWN_DIR, file)
        img = face_recognition.load_image_file(path)

        img = cv2.resize(img, (0, 0), fx=RESIZE_SCALE, fy=RESIZE_SCALE)

        encodings = face_recognition.face_encodings(img)

        if len(encodings) == 1:
            return encodings[0], os.path.splitext(file)[0]

        return None

    except:
        return None


# ---------- Load + Update Encodings ----------
def load_known_faces():
    known_faces, known_names = [], []

    if os.path.exists(ENCODING_FILE):
        with open(ENCODING_FILE, "rb") as f:
            known_faces, known_names = pickle.load(f)

    existing = set(known_names)

    files = [f for f in os.listdir(KNOWN_DIR) if f.lower().endswith((".jpg",".png",".jpeg"))]
    files = [f for f in files if os.path.splitext(f)[0] not in existing]

    if files:
        with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            results = list(executor.map(encode_image, files))

        for r in results:
            if r:
                known_faces.append(r[0])
                known_names.append(r[1])

        os.makedirs(os.path.dirname(ENCODING_FILE), exist_ok=True)
        with open(ENCODING_FILE, "wb") as f:
            pickle.dump((known_faces, known_names), f)

    return known_faces, known_names


# ---------- Build or Load FAISS ----------
def get_faiss_index(known_faces):
    known_faces_np = np.array(known_faces).astype("float32")

    if os.path.exists(FAISS_INDEX_FILE):
        return faiss.read_index(FAISS_INDEX_FILE)

    index = faiss.IndexFlatL2(128)
    index.add(known_faces_np)

    faiss.write_index(index, FAISS_INDEX_FILE)
    return index


# ---------- Face Recognition ----------
def recognize_face(input_image_file):
    start_time = time.time()

    known_faces, known_names = load_known_faces()

    if len(known_faces) == 0:
        return {"status": "no_known_faces"}

    index = get_faiss_index(known_faces)

    input_image = face_recognition.load_image_file(input_image_file)

    face_locations = face_recognition.face_locations(input_image, model="hog")
    if not face_locations:
        return {"status": "no_face_detected"}

    encodings = face_recognition.face_encodings(input_image, face_locations)
    if not encodings:
        return {"status": "encoding_failed"}

    matched_person = None
    matched_confidence = None

    for enc in encodings:
        enc = np.array([enc]).astype("float32")

        D, I = index.search(enc, 1)
        best_distance = D[0][0]
        best_index = I[0][0]

        if best_distance < MATCH_THRESHOLD:
            matched_person = known_names[best_index]
            matched_confidence = round(1 - best_distance, 3)
            break

    return {
        "status": "matched" if matched_person else "not_matched",
        "person": matched_person,
        "confidence": matched_confidence,
        "time_taken_sec": round(time.time() - start_time, 3)
    }


# ---------- CLI ----------
if __name__ == "__main__":
    import sys

    if len(sys.argv) > 1:
        print(json.dumps(recognize_face(sys.argv[1]), indent=2))
    else:
        print("Usage: python script.py image.jpg")


🖼️ Found 415 dataset images
🆕 New images to encode: 415
📸 Loading: known_faces/dataset_imgs\100_10.jpg
📸 Loading: known_faces/dataset_imgs\101_11.jpg
🔍 Faces found: 1
✅ Encoded: 100_10.jpg
📸 Loading: known_faces/dataset_imgs\102_11.jpg
📸 Loading: known_faces/dataset_imgs\103_11.jpg
📸 Loading: known_faces/dataset_imgs\104_11.jpg
🔍 Faces found: 1
✅ Encoded: 101_11.jpg
📸 Loading: known_faces/dataset_imgs\105_11.jpg
📸 Loading: known_faces/dataset_imgs\106_11.jpg
🔍 Faces found: 1
✅ Encoded: 102_11.jpg
📸 Loading: known_faces/dataset_imgs\107_11.jpg
🔍 Faces found: 1
✅ Encoded: 105_11.jpg
🔍 Faces found: 1
✅ Encoded: 106_11.jpg
🔍 Faces found: 1
✅ Encoded: 104_11.jpg
🔍 Faces found: 1
✅ Encoded: 103_11.jpg
📸 Loading: known_faces/dataset_imgs\108_11.jpg📸 Loading: known_faces/dataset_imgs\109_11.jpg
📸 Loading: known_faces/dataset_imgs\10_1.jpg

📸 Loading: known_faces/dataset_imgs\110_11.jpg
📸 Loading: known_faces/dataset_imgs\111_12.jpg
📸 Loading: known_faces/dataset_imgs\112_12.jpg
📸 Loading: know

: 